# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [36]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [37]:
diy_lu(np.array([[1, 1], [1, 2]]))

(array([[1., 0.],
        [1., 1.]]), array([[1., 1.],
        [0., 1.]]))

In [22]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [23]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [24]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [25]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [26]:
np.linalg.matrix_rank(a1)

6

In [27]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [28]:
n = len(a)
flag1 = 0
flag2 = 0
for i in range(n):
    for j in range(n):
        minor = np.linalg.det(np.delete(np.delete(a, i, axis = 0), j, axis = 1))
        if minor == 0:
            flag1 = 1
            print('a имеет нулевой минор')
            
for i in range(n):
    for j in range(n):
        minor = np.linalg.det(np.delete(np.delete(a1, i, axis = 0), j, axis = 1))
        if minor == 0:
            flag2 = 1
            print('a1 имеет нулевой минор')
            
if flag1 == 0:
    print('с a все в порядке')
if flag2 == 0:
    print('с a1 все в порядке')

с a все в порядке
с a1 все в порядке


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [356]:
def PLU(a):
     #делаем все то же самое, только еще вставляем вращения
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    pivmat = np.eye(N) # матрица вращения
    for j in range(N-1):   # вместо максимума просто берем первый ненулевой элемент
        for i in range(j, N):
                if u[i, j] != 0:
                    target = i
                    break
        #print(target)
        elementary_piv = np.eye(N) #элементарное вращение
        elementary_piv[j, j] = 0
        elementary_piv[target, target] = 0
        elementary_piv[j, target] = 1
        elementary_piv[target, j] = 1 #наверно есть какой-нибудь метод сделать это быстро, но дольше искать,
        # чем написать вот так вот
        u[[j, target],:] = u[[target, j],:]
        pivmat = elementary_piv@pivmat
        
        
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam 
        #print(u)
    return L, u, pivmat

In [357]:
def recomp(L, u, P):
    return P@L@u

In [358]:
recomp(*PLU(a1)) -a1

array([[ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  2.220e-16, -1.110e-16,
        -1.665e-16],
       [ 0.000e+00,  0.000e+00,  2.220e-16, -5.551e-17, -1.665e-16,
        -1.665e-16],
       [ 0.000e+00,  0.000e+00, -1.110e-16, -1.665e-16,  1.665e-16,
         5.551e-17],
       [ 0.000e+00,  0.000e+00, -1.665e-16, -1.665e-16,  5.551e-17,
         0.000e+00]])

In [359]:
recomp(*PLU(a)) - a

array([[ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  2.220e-16, -1.110e-16,
        -1.665e-16],
       [ 0.000e+00,  0.000e+00,  2.220e-16, -5.551e-17, -1.665e-16,
        -1.665e-16],
       [ 0.000e+00,  0.000e+00, -1.110e-16,  2.776e-16, -2.776e-16,
         5.551e-17],
       [ 0.000e+00,  0.000e+00, -1.665e-16, -1.665e-16,  5.551e-17,
         0.000e+00]])